In [3]:
from google.colab import files

In [4]:
acs_data = files.upload()

Saving acs2017_census_tract_data.csv to acs2017_census_tract_data.csv


In [5]:
covid_data = files.upload()

Saving COVID_county_data.csv to COVID_county_data.csv


In [6]:
import pandas as pd

# Load the ACS data
acs_data = pd.read_csv('acs2017_census_tract_data.csv')
print(acs_data.head(10))

# Load the COVID data
covid_data = pd.read_csv('COVID_county_data.csv')
print(covid_data.head(10))


      TractId    State          County  TotalPop   Men  Women  Hispanic  \
0  1001020100  Alabama  Autauga County      1845   899    946       2.4   
1  1001020200  Alabama  Autauga County      2172  1167   1005       1.1   
2  1001020300  Alabama  Autauga County      3385  1533   1852       8.0   
3  1001020400  Alabama  Autauga County      4267  2001   2266       9.6   
4  1001020500  Alabama  Autauga County      9965  5054   4911       0.9   
5  1001020600  Alabama  Autauga County      3620  1765   1855       3.0   
6  1001020700  Alabama  Autauga County      3420  1459   1961       4.0   
7  1001020801  Alabama  Autauga County      2913  1495   1418       4.2   
8  1001020802  Alabama  Autauga County     11333  5488   5845       1.4   
9  1001020900  Alabama  Autauga County      6167  3111   3056       1.0   

   White  Black  Native  ...  Walk  OtherTransp  WorkAtHome  MeanCommute  \
0   86.3    5.2     0.0  ...   0.5          0.0         2.1         24.5   
1   41.6   54.5     0.

In [7]:
# Transforming ACS DATA
# Select relevant columns
acs_data = acs_data[['County', 'State', 'TotalPop', 'Poverty', 'IncomePerCap']]

# Aggregate data to the county level
county_info = acs_data.groupby(['County', 'State']).agg(
    Population=('TotalPop', 'sum'),
    Poverty=('Poverty', 'mean'),
    PerCapitaIncome=('IncomePerCap', 'mean')
).reset_index()

# Add a unique ID for each county
county_info['ID'] = county_info.index + 1

# Display the County_info DataFrame
county_info.head()




,County,State,Population,Poverty,PerCapitaIncome,ID
0,Abbeville County,South Carolina,24788,22.183333,19402.833333,1
1,Acadia Parish,Louisiana,62607,21.933333,21454.250000,2
2,Accomack County,Virginia,32840,20.112500,24769.750000,3
3,Ada County,Idaho,435117,12.608475,32672.661017,4
4,Adair County,Iowa,7192,9.566667,28840.666667,5


In [8]:
# Extract required rows for the table
required_counties = ['Loudoun County', 'Washington County', 'Harlan County', 'Malheur County']
required_states = ['Virginia', 'Oregon', 'Kentucky', 'Oregon']
table_data = county_info[county_info.apply(lambda row: row['County'] in required_counties and row['State'] in required_states, axis=1)]

print(table_data)


                 County     State  Population    Poverty  PerCapitaIncome  \
1234      Harlan County  Kentucky       27548  33.318182     16010.363636   
1759     Loudoun County  Virginia      374558   3.884375     50391.015625   
1822     Malheur County    Oregon       30421  24.414286     17966.428571   
3035  Washington County  Kentucky       11999  16.900000     22354.666667   
3046  Washington County    Oregon      572071  10.446154     34970.817308   
3053  Washington County  Virginia       54457  14.715385     26276.538462   

        ID  
1234  1235  
1759  1760  
1822  1823  
3035  3036  
3046  3047  
3053  3054  


In [9]:
most_populous_county = county_info.loc[county_info['Population'].idxmax()]
least_populous_county = county_info.loc[county_info['Population'].idxmin()]

print(f"Most populous county in the USA: {most_populous_county['County']}, {most_populous_county['State']} with population {most_populous_county['Population']}")
print(f"Least populous county in the USA: {least_populous_county['County']}, {least_populous_county['State']} with population {least_populous_county['Population']}")


Most populous county in the USA: Los Angeles County, California with population 10105722
Least populous county in the USA: Loving County, Texas with population 74


In [10]:


# Convert 'date' column to datetime type
covid_data['date'] = pd.to_datetime(covid_data['date'])

# Extract month and year from the 'date' column
covid_data['month_year'] = covid_data['date'].dt.to_period('M')

# Group by county and month_year, and sum the cases and deaths
covid_monthly = covid_data.groupby(['county', 'state', 'fips', 'month_year']).agg({'cases': 'sum', 'deaths': 'sum'}).reset_index()

# Add an 'ID' column starting from 1
covid_monthly['ID'] = range(1, len(covid_monthly) + 1)

# Set 'ID' as the index
covid_monthly.set_index('ID', inplace=True)

# Display the COVID_monthly DataFrame
print(covid_monthly[covid_monthly['county'] =='Malheur'])
print(covid_monthly)
malheur_data = covid_monthly[covid_monthly['county'] =='Malheur']


        county   state     fips month_year  cases  deaths
ID                                                       
20757  Malheur  Oregon  41045.0    2020-03      1     0.0
20758  Malheur  Oregon  41045.0    2020-04    125     0.0
20759  Malheur  Oregon  41045.0    2020-05    626     0.0
20760  Malheur  Oregon  41045.0    2020-06   1447    24.0
20761  Malheur  Oregon  41045.0    2020-07  12773   130.0
20762  Malheur  Oregon  41045.0    2020-08  28163   459.0
20763  Malheur  Oregon  41045.0    2020-09  43150   693.0
20764  Malheur  Oregon  41045.0    2020-10  56398  1061.0
20765  Malheur  Oregon  41045.0    2020-11  65787  1174.0
20766  Malheur  Oregon  41045.0    2020-12  82916  1465.0
20767  Malheur  Oregon  41045.0    2021-01  96297  1627.0
20768  Malheur  Oregon  41045.0    2021-02  65951  1137.0
          county           state     fips month_year  cases  deaths
ID                                                                 
1      Abbeville  South Carolina  45001.0    2020-03

In [11]:
print(county_info)

                County           State  Population    Poverty  \
0     Abbeville County  South Carolina       24788  22.183333   
1        Acadia Parish       Louisiana       62607  21.933333   
2      Accomack County        Virginia       32840  20.112500   
3           Ada County           Idaho      435117  12.608475   
4         Adair County            Iowa        7192   9.566667   
...                ...             ...         ...        ...   
3215       Yuma County         Arizona      204281  19.357692   
3216       Yuma County        Colorado       10109  14.350000   
3217     Zapata County           Texas       14415  32.566667   
3218     Zavala County           Texas       12152  41.375000   
3219    Ziebach County    South Dakota        2837  43.100000   

      PerCapitaIncome    ID  
0        19402.833333     1  
1        21454.250000     2  
2        24769.750000     3  
3        32672.661017     4  
4        28840.666667     5  
...               ...   ...  
3215     

In [12]:
# Filter for specific months
required_months = ['2020-08', '2021-01', '2021-02']
malheur_data_filtered = malheur_data[malheur_data['month_year'].astype(str).isin(required_months)]

# Display the results
print(malheur_data_filtered[['county', 'state', 'month_year', 'cases', 'deaths']])

        county   state month_year  cases  deaths
ID                                              
20762  Malheur  Oregon    2020-08  28163   459.0
20767  Malheur  Oregon    2021-01  96297  1627.0
20768  Malheur  Oregon    2021-02  65951  1137.0


In [13]:
print(county_info.head(5))
print(covid_monthly.head(5))


             County           State  Population    Poverty  PerCapitaIncome  \
0  Abbeville County  South Carolina       24788  22.183333     19402.833333   
1     Acadia Parish       Louisiana       62607  21.933333     21454.250000   
2   Accomack County        Virginia       32840  20.112500     24769.750000   
3        Ada County           Idaho      435117  12.608475     32672.661017   
4      Adair County            Iowa        7192   9.566667     28840.666667   

   ID  
0   1  
1   2  
2   3  
3   4  
4   5  
       county           state     fips month_year  cases  deaths
ID                                                              
1   Abbeville  South Carolina  45001.0    2020-03     32     0.0
2   Abbeville  South Carolina  45001.0    2020-04    430     0.0
3   Abbeville  South Carolina  45001.0    2020-05   1080     0.0
4   Abbeville  South Carolina  45001.0    2020-06   2187     0.0
5   Abbeville  South Carolina  45001.0    2020-07   5959    65.0


In [15]:
# Aggregate COVID data to the county level
covid_summary = covid_monthly.groupby('ID').agg(
    TotalCases=('cases', 'sum'),
    TotalDeaths=('deaths', 'sum')
).reset_index()

# Merge with County_info
covid_summary = covid_summary.merge(county_info, on='ID')

# Calculate additional columns
covid_summary['TotalCasesPer100K'] = covid_summary['TotalCases'] / (covid_summary['Population'] / 100000)
covid_summary['TotalDeathsPer100K'] = covid_summary['TotalDeaths'] / (covid_summary['Population'] / 100000)

# Display the COVID_summary DataFrame
covid_summary.head()



,ID,TotalCases,TotalDeaths,County,State,Population,Poverty,PerCapitaIncome,TotalCasesPer100K,TotalDeathsPer100K
0,1,32,0.0,Abbeville County,South Carolina,24788,22.183333,19402.833333,129.094723,0.00000
1,2,430,0.0,Acadia Parish,Louisiana,62607,21.933333,21454.250000,686.824157,0.00000
2,3,1080,0.0,Accomack County,Virginia,32840,20.112500,24769.750000,3288.672351,0.00000
3,4,2187,0.0,Ada County,Idaho,435117,12.608475,32672.661017,502.623432,0.00000
4,5,5959,65.0,Adair County,Iowa,7192,9.566667,28840.666667,82855.951057,903.78198


In [16]:
# Extract required rows for the table
required_counties = ['Washington County', 'Malheur County', 'Loudoun County', 'Harlan County']
required_states = ['Oregon', 'Oregon', 'Virginia', 'Kentucky']
table_data = covid_summary[covid_summary.apply(lambda row: row['County'] in required_counties and row['State'] in required_states, axis=1)]

print(table_data)


        ID  TotalCases  TotalDeaths             County     State  Population  \
1234  1235         118          2.0      Harlan County  Kentucky       27548   
1759  1760        1178         12.0     Loudoun County  Virginia      374558   
1822  1823       33048        601.0     Malheur County    Oregon       30421   
3035  3036        9513         75.0  Washington County  Kentucky       11999   
3046  3047       76191        890.0  Washington County    Oregon      572071   
3053  3054        2823         93.0  Washington County  Virginia       54457   

        Poverty  PerCapitaIncome  TotalCasesPer100K  TotalDeathsPer100K  
1234  33.318182     16010.363636         428.343255            7.260055  
1759   3.884375     50391.015625         314.504029            3.203776  
1822  24.414286     17966.428571      108635.482068         1975.608954  
3035  16.900000     22354.666667       79281.606801          625.052088  
3046  10.446154     34970.817308       13318.451731          155.5750

In [17]:
# Extract required rows for the table
required_counties = ['Washington County', 'Malheur County', 'Loudoun County', 'Harlan County']
required_states = ['Oregon', 'Oregon', 'Virginia', 'Kentucky']
table_data = covid_summary[covid_summary.apply(lambda row: row['County'] in required_counties and row['State'] in required_states, axis=1)]

print(table_data)


        ID  TotalCases  TotalDeaths             County     State  Population  \
1234  1235         118          2.0      Harlan County  Kentucky       27548   
1759  1760        1178         12.0     Loudoun County  Virginia      374558   
1822  1823       33048        601.0     Malheur County    Oregon       30421   
3035  3036        9513         75.0  Washington County  Kentucky       11999   
3046  3047       76191        890.0  Washington County    Oregon      572071   
3053  3054        2823         93.0  Washington County  Virginia       54457   

        Poverty  PerCapitaIncome  TotalCasesPer100K  TotalDeathsPer100K  
1234  33.318182     16010.363636         428.343255            7.260055  
1759   3.884375     50391.015625         314.504029            3.203776  
1822  24.414286     17966.428571      108635.482068         1975.608954  
3035  16.900000     22354.666667       79281.606801          625.052088  
3046  10.446154     34970.817308       13318.451731          155.5750

In [18]:

oregon_counties = covid_summary[covid_summary['State'] == 'Oregon']
r_oregon_cases_poverty = oregon_counties['TotalCasesPer100K'].corr(oregon_counties['Poverty'])
r_oregon_deaths_poverty = oregon_counties['TotalDeathsPer100K'].corr(oregon_counties['Poverty'])
r_oregon_cases_income = oregon_counties['TotalCasesPer100K'].corr(oregon_counties['PerCapitaIncome'])
r_oregon_deaths_income = oregon_counties['TotalDeathsPer100K'].corr(oregon_counties['PerCapitaIncome'])


r_usa_cases_poverty = covid_summary['TotalCasesPer100K'].corr(covid_summary['Poverty'])
r_usa_deaths_poverty = covid_summary['TotalDeathsPer100K'].corr(covid_summary['Poverty'])
r_usa_cases_income = covid_summary['TotalCasesPer100K'].corr(covid_summary['PerCapitaIncome'])
r_usa_deaths_income = covid_summary['TotalDeathsPer100K'].corr(covid_summary['PerCapitaIncome'])

print(f"Correlation for Oregon counties (cases vs. poverty): {r_oregon_cases_poverty}")
print(f"Correlation for Oregon counties (deaths vs. poverty): {r_oregon_deaths_poverty}")
print(f"Correlation for Oregon counties (cases vs. income): {r_oregon_cases_income}")
print(f"Correlation for Oregon counties (deaths vs. income): {r_oregon_deaths_income}")
print(f"Correlation for all USA counties (cases vs. poverty): {r_usa_cases_poverty}")
print(f"Correlation for all USA counties (deaths vs. poverty): {r_usa_deaths_poverty}")
print(f"Correlation for all USA counties (cases vs. income): {r_usa_cases_income}")
print(f"Correlation for all USA counties (deaths vs. income): {r_usa_deaths_income}")


Correlation for Oregon counties (cases vs. poverty): -0.05647650139352764
Correlation for Oregon counties (deaths vs. poverty): 0.10043666526621915
Correlation for Oregon counties (cases vs. income): 0.204117283718057
Correlation for Oregon counties (deaths vs. income): -0.05827613303995275
Correlation for all USA counties (cases vs. poverty): -0.029367080872852284
Correlation for all USA counties (deaths vs. poverty): -0.020898132620350507
Correlation for all USA counties (cases vs. income): 0.053454007189515075
Correlation for all USA counties (deaths vs. income): 0.045363536141161315
